# MLIP hackathon starter notebook

In this challenge, we train a Machine Learning Interatomic Potential (MLIP) model using the [`mlip`](https://github.com/instadeepai/mlip) library.

If you run this on Google Colab, make sure to select a GPU runtime (Runtime > Change runtime type > Hardware accelerator > GPU).

In [212]:
#!pip install mlip "jax[cuda12]"

In [213]:
#!pip install wandb

**Install, required imports, and logging setup**


In [214]:
# import wandb
# !wandb login

In [215]:
import os
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt

# For dataset loading
from mlip.data import GraphDatasetBuilder, ExtxyzReader

# For model
from mlip.models import Mace, Nequip, Visnet, ForceField

# For optimizer
import optax

# For loss function
from mlip.models.loss import MSELoss

# For training
from mlip.training import TrainingLoop
from mlip.models.model_io import save_model_to_zip, load_model_from_zip
from mlip.models.params_loading import load_parameters_from_checkpoint

# For checkpointing
from mlip.training import TrainingIOHandler, log_metrics_to_line
from mlip.training.training_io_handler import LogCategory

# Set up logging
logging.basicConfig(level=logging.INFO, force=True, format='%(levelname)s - %(message)s')

# Set dedicated logging for mlip. Set this to logging.DEBUG to see more detailed logs.
logging.getLogger("mlip").setLevel(logging.INFO)

Let's also check what device we are using:

In [216]:
import jax

print(jax.devices())

[CudaDevice(id=0)]


## 1. Preparing a dataset

For this example, we train on configurations of a molecule called [3-(benzyloxy)pyridin-2-amine](https://pubchem.ncbi.nlm.nih.gov/substance/854545) (molecular formula `C12H12N2O`, abbreviated as `3BPA`) sampled with Molecular Dynamics at a temperature of 300 Kelvin.

It's molecular structure consists of a **pyridin-2-amine core**:

 * A **pyridine ring** (six-membered aromatic ring with one nitrogen atom).

 * An **amino group (-NH₂)** at the 2-position of the pyridine ring (adjacent to the nitrogen).

Benzyloxy substituent at position 3:

 * A **benzyloxy group (-OCH₂C₆H₅)** is attached to the 3-position of the pyridine ring.

 * This consists of a **methylene bridge (–CH₂–)** bonded to a **phenyl ring (C₆H₅)**, connected via an **ether linkage (–O–)**.

![3BPA](https://go.drugbank.com/structures/DB02352/thumb.svg)

The data processing **is a two step process**:

1. **We read the data from disk into [`ChemicalSystem`](https://instadeepai.github.io/mlip/api_reference/data/chemical_system.html) objects**. This is done by a "reader", and since the dataset is stored in extended xyz format, it can be read with the [`ExtxyzReader`](https://instadeepai.github.io/mlip/api_reference/data/chemical_systems_readers/extxyz_reader.html). The *mlip* library also includes a HDF5 format reader: [`Hdf5Reader`](https://instadeepai.github.io/mlip/api_reference/data/chemical_systems_readers/hdf5_reader.html).

In [217]:
%%bash
mkdir -p data

[ -f data/test_public.xyz ] || wget -P data https://raw.githubusercontent.com/BioGeek/hackathon_IndabaX_2025_mlip/refs/heads/main/data/test_public.xyz
[ -f data/train.xyz ] || wget -P data https://raw.githubusercontent.com/BioGeek/hackathon_IndabaX_2025_mlip/refs/heads/main/data/train.xyz
[ -f data/validation.xyz ] || wget -P data https://raw.githubusercontent.com/BioGeek/hackathon_IndabaX_2025_mlip/refs/heads/main/data/validation.xyz

In [218]:
reader = ExtxyzReader(
    ExtxyzReader.Config(
        train_dataset_paths="data/train.xyz",
        valid_dataset_paths="data/validation.xyz")
)

2. **We process these [`ChemicalSystem`](https://instadeepai.github.io/mlip/api_reference/data/chemical_system.html) objects into graphs.** This process uses the class [`GraphDatasetBuilder`](https://instadeepai.github.io/mlip/api_reference/data/graph_dataset_builder.html) which offers some degree of customisation through its [config class](https://instadeepai.github.io/mlip/api_reference/data/dataset_configs.html#mlip.data.configs.GraphDatasetBuilderConfig).



In [219]:
builder_config = GraphDatasetBuilder.Config(
    graph_cutoff_angstrom=5.0,
    batch_size=16,
)

builder = GraphDatasetBuilder(reader, builder_config)
builder.prepare_datasets() # This step is required to compute all dataset information (used later on by most MLIP model)

train_set, validation_set, _ = builder.get_splits()

Graph creation:   0%|          | 0/500 [00:00<?, ?it/s]

valid graph creation:   0%|          | 0/100 [00:00<?, ?it/s]

test graph creation: 0it [00:00, ?it/s]

INFO - Starting to compute mandatory dataset statistics: this may take some time...


Average number of neighbors computation:   0%|          | 0/500 [00:00<?, ?it/s]

INFO - Processed 10% of data
INFO - Processed 20% of data
INFO - Processed 30% of data
INFO - Processed 40% of data
INFO - Processed 50% of data
INFO - Processed 60% of data
INFO - Processed 70% of data
INFO - Processed 80% of data
INFO - Processed 90% of data
INFO - Processed 100% of data


More information can be found in the [deep-dive on data processing](https://instadeepai.github.io/mlip/user_guide/data_processing.html)  in our documentation for more details.

We can now **print some statistics about our dataset** along with the [`DatasetInfo`](https://instadeepai.github.io/mlip/api_reference/data/dataset_info.html) object that will be required for downstream tasks. The **dataset info** holds all the hyperparameters of the models that are directly derived from the dataset or its processing, e.g., the cutoff distance to determine the graph edges.

In [220]:
print("Dataset info:", builder.dataset_info)
print("Number of batches in train set:", len(train_set))
print("Number of batches in validation set:", len(validation_set))

Dataset info: Atomic Energies: {'H': -723.2941604798493, 'C': -723.2941604798525, 'N': -120.54902674664157, 'O': -60.27451337332064}, Avg. num. neighbors: 16.72, Avg. r_min: 1.00, Graph cutoff distance: 5.0
Number of batches in train set: 32
Number of batches in validation set: 7


The atomic energies and forces are stored in the `energy` and `forces` attributes of the [`ChemicalSystem`](https://instadeepai.github.io/mlip/api_reference/data/chemical_system.html) objects, respectively. The energies are in [eV](https://en.wikipedia.org/wiki/Electronvolt), and the forces are in eV/[Å](https://en.wikipedia.org/wiki/Angstrom).

## 2. Preparing a training loop

To start training, we first need to prepare some prerequisites. These are, as for all ML models:
- A **model architecture**,
- An **optimizer**, and
- A **loss function**

We start with the **model architecture**:

We can use one of the pre-defined models in the *mlip* library, such as MACE, NequIP, or ViSNet. These models are designed to handle molecular graphs and can be configured with various hyperparameters.

For this tutorial, we provide the initialization code for MACE, NequIP and ViSnet, but commented out two of them. For all the hyperparameters available, see the documentations of the [MACE config](https://instadeepai.github.io/mlip/api_reference/models/mace.html#mlip.models.mace.config.MaceConfig), the [NequIP config](https://instadeepai.github.io/mlip/api_reference/models/nequip.html#mlip.models.nequip.config.NequipConfig), and the [ViSNet config](https://instadeepai.github.io/mlip/api_reference/models/visnet.html#mlip.models.visnet.config.VisnetConfig).

The model creation process includes two steps:
* 1: the creation of the MLIP network and
* 2:the creation  of the force field.

See our [deep-dive on models](https://instadeepai.github.io/mlip/user_guide/models.html) for a detailed explanation of this pattern.

**Hint**: try some of the other pre-defined models by uncommenting the lines below and running the cell again. You can also try to change some of the hyperparameters in the config classes, e.g., the number of layers or channels

In [221]:
# run = wandb.init(
#     # Set the wandb entity where your project will be logged (generally your team name).
#     entity="cyberCharl",
#     # Set the wandb project where this run will be logged.
#     project="Instadeep Hackathon",
#     # Track hyperparameters and run metadata.
#     config={
#         "epochs": 100,
#         "node_irreps": "4x0e + 4x0o + 4x1o + 4x1e + 4x2e + 4x2o",
#         "num_layers": 4,
#         "weight_decay": 1e-5,
#         "grad_norm": 50,
#         "num_gradient_accumulation_steps": 1,
#         "init_learning_rate": 5e-4,  # Start a bit higher
#         "peak_learning_rate": 2e-3,  # Target higher peak
#         "final_learning_rate": 1e-4,  # Decay to a lower final LR
#         "warmup_steps": 10000,  # Adjusted for your dataset size and epoch count (approx 5 epochs)
#         "transition_steps": 1840000,
#         "energy_weight": 500.0
#     },
# )

In [222]:
sweep_config = {
    'method': 'bayes',  # or 'grid', 'random'
    'metric': {
        'name': 'val_loss',  # This should match your logged metric
        'goal': 'minimize'
    },
    'parameters': {
        'num_layers': {'values': [2, 4, 6]},
        'init_learning_rate': {'min': 1e-4, 'max': 5e-3},
        'peak_learning_rate': {'min': 1e-3, 'max': 5e-3},
        'final_learning_rate': {'min': 1e-5, 'max': 1e-3},
        'weight_decay': {'min': 1e-6, 'max': 1e-3},
        'energy_weight': {'values': [100.0, 500.0, 1000.0]}
        # Add more as needed
    }
}

The force field will be the essential object required for the

---

training below, as well as for running MD simulations.

In [ ]:
import wandb
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="cybercharl-ai-safety-south-africa",
    # Set the wandb project where this run will be logged.
    project="instadeep-hackathon-2025",
    # Track hyperparameters and run metadata.
    config={
        "epochs": 100,
        "node_irreps": "4x0e + 4x0o + 4x1o + 4x1e + 4x2e + 4x2o",
        "num_layers": 4,
        "weight_decay": 1e-5,
        "grad_norm": 50,
        "num_gradient_accumulation_steps": 1,
        "init_learning_rate": 5e-4,  # Start a bit higher
        "peak_learning_rate": 2e-3,  # Target higher peak
        "final_learning_rate": 1e-4,  # Decay to a lower final LR
        "warmup_steps": 10000,  # Adjusted for your dataset size and epoch count (approx 5 epochs)
        "transition_steps": 1840000,
        "energy_weight": 500.0
    }
)

def train():
    # 1. Initialize wandb run
    run = wandb.init() # Commented out this line as the sweep will handle init
    config = run.config

    # 2. Build model with sweep hyperparameters
    mlip_network = Nequip(
        Nequip.Config(
            node_irreps="4x0e + 4x0o + 4x1o + 4x1e + 4x2e + 4x2o",  # or sweep this too
            num_layers=config.num_layers,
        ),
        builder.dataset_info,
    )
    force_field = ForceField.from_mlip_network(mlip_network)

    # 3. Optimizer config
    from mlip.training.optimizer_config import OptimizerConfig # Import here
    optimizer_config = OptimizerConfig(
        apply_weight_decay_mask=True,
        weight_decay=config.weight_decay,
        grad_norm=50,  # or config.grad_norm
        num_gradient_accumulation_steps=1,
        init_learning_rate=config.init_learning_rate,
        peak_learning_rate=config.peak_learning_rate,
        final_learning_rate=config.final_learning_rate,
        warmup_steps=10000,  # or config.warmup_steps
        transition_steps=1840000,  # or config.transition_steps
    )
    from mlip.training.optimizer import get_default_mlip_optimizer # Import here
    optimizer = get_default_mlip_optimizer(optimizer_config)

    # 4. Loss and training config
    from mlip.models.loss import MSELoss # Import here
    loss = MSELoss()
    from mlip.training import TrainingLoop # Import here
    training_config = TrainingLoop.Config(
        num_epochs=100,  # or config.epochs
        energy_weight=config.energy_weight,
        force_weight=0,  # or config.force_weight
    )

    # 5. IO handler and logger
    from mlip.training import TrainingIOHandler, log_metrics_to_line # Import here
    io_handler = TrainingIOHandler(
        TrainingIOHandler.Config(local_model_output_dir="training/model_training")
    )
    io_handler.attach_logger(log_metrics_to_line)

    # Custom logger to log val_loss to wandb
    from mlip.training.training_io_handler import LogCategory # Import here
    def wandb_logger(category, to_log, epoch_number):
        if category == LogCategory.EVAL_METRICS:
          run.log(to_log) # Log val_loss with step


    io_handler.attach_logger(wandb_logger)

    # 6. Training loop
    training_loop = TrainingLoop(
        train_dataset=train_set,
        validation_dataset=validation_set,
        force_field=force_field,
        loss=loss,
        optimizer=optimizer,
        config=training_config,
        io_handler=io_handler,
    )
    training_loop.run()

    # Save the best model
    save_model_to_zip(f"training/model_{wandb.run.id}.zip", training_loop.best_model)

    # Run inference and save CSV
    from ase.io import read as ase_read
    test_data = "data/test_public.xyz"
    structures = ase_read(test_data, index=":")

    from mlip.inference import run_batched_inference
    predictions = run_batched_inference(structures, training_loop.best_model, batch_size=8)
    energies = np.array([prediction.energy for prediction in predictions])

    df = pd.DataFrame({
        'ID': np.arange(len(energies)),
        'energies': energies
    })

    csv_name = f"{wandb.run.id}_submission.csv"
    df.to_csv(csv_name, index=False)
    wandb.save(csv_name)  # Uploads to wandb for easy download

sweep_id = wandb.sweep(sweep_config, project="instadeep-hackathon-2025")
wandb.agent(sweep_id, function=train)

ERROR - Task was destroyed but it is pending!
task: <Task cancelling name='Task-86786' coro=<Event.wait() running at /usr/lib/python3.11/asyncio/locks.py:213> wait_for=<Future cancelled>>
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 156uxkmm
Sweep URL: https://wandb.ai/cybercharl-ai-safety-south-africa/instadeep-hackathon-2025/sweeps/156uxkmm


wandb: Agent Starting Run: qubguh2d with config:
wandb: 	energy_weight: 500
wandb: 	final_learning_rate: 0.00027571455565362585
wandb: 	init_learning_rate: 0.000585894064037785
wandb: 	num_layers: 2
wandb: 	peak_learning_rate: 0.0028892082906843974
wandb: 	weight_decay: 0.000806450734688251


INFO - Cleaning up existing temporary directories at /content/training/model_training/model.
INFO - Number of parameters: 29098
INFO - Number of parameters in optimizer: 116398
INFO - Starting training loop...
INFO - Validation: Loss = 87.034 | Mae e = 3.000 | Mae f = 1.389 | Rmse e = 3.066 | Rmse f = 1.866
INFO - Best model: Loss = 87.034 | Best epoch = 0


here??


...

here??


INFO - ------------ Epoch 42 ------------
INFO - Training:   Loss = 0.497 | Gradient norm = 9.493 | Param update norm = 0.033 | Runtime in seconds = 1.416
INFO - Validation: Loss = 11.905 | Mae e = 2.370 | Mae f = 0.446 | Rmse e = 2.384 | Rmse f = 0.690
INFO - Saving checkpoint at epoch 42...
INFO - Best model: Loss = 11.905 | Best epoch = 42


here??


INFO - ------------ Epoch 43 ------------
INFO - Training:   Loss = 0.456 | Gradient norm = 4.652 | Param update norm = 0.025 | Runtime in seconds = 1.392
INFO - Validation: Loss = 11.714 | Mae e = 2.293 | Mae f = 0.442 | Rmse e = 2.308 | Rmse f = 0.684
INFO - Saving checkpoint at epoch 43...
INFO - Best model: Loss = 11.714 | Best epoch = 43


here??


In [ ]:
# force_field = ForceField.from_mlip_network(mlip_network)

Next, we **create an optimizer**:

The *mlip* library is set up so that you can use any [`optax`](https://github.com/google-deepmind/optax) optimizer you like, here we've chosen for [`optax.adam`](https://optax.readthedocs.io/en/latest/api/optimizers.html#optax.adam).

**Hint:** Also try an optimizer specialized for MLIP models (see [this](https://instadeepai.github.io/mlip/api_reference/training/optimizer.html#mlip.training.optimizer.get_default_mlip_optimizer) part of the documentation for more details).

In [ ]:
# from mlip.training.optimizer import get_default_mlip_optimizer
# from mlip.training.optimizer_config import OptimizerConfig

# # Configure the specialized MLIP optimizer
# optimizer_config = OptimizerConfig(
#     apply_weight_decay_mask=True,
#     weight_decay= run.config["weight_decay"],
#     grad_norm=run.config["grad_norm"],
#     num_gradient_accumulation_steps=run.config["num_gradient_accumulation_steps"],
#     init_learning_rate=run.config["init_learning_rate"],
#     peak_learning_rate=run.config["peak_learning_rate"],
#     final_learning_rate=run.config["final_learning_rate"],
#     warmup_steps=run.config["warmup_steps"],
#     transition_steps=run.config["transition_steps"]
# )

# optimizer = get_default_mlip_optimizer(optimizer_config)

# print("Loss function:", loss)
# print("Optimizer configuration:", optimizer_config)
# print("Optimizer:", optimizer)

For the **loss**:

We use a Mean-Squared-Error (MSE) loss, that by default uses a weighting factor of 25.0 for MSE of forces, 1.0 for MSE of energies, and zero for MSE of stress (which is not available in this dataset). See [this](https://instadeepai.github.io/mlip/user_guide/training.html#loss) part of the documentation for more information on further options such as alternative loss functions and how to create a weight flip schedule between energy and forces.

In [ ]:
# loss = MSELoss()

# energy_weight_schedule = optax.piecewise_constant_schedule(run.config["energy_weight"])

We can now set up a **custom I/O handler** with checkpointing for training:

The I/O handler class is documented [here](https://instadeepai.github.io/mlip/api_reference/training/training_io_handling.html). Also check out [this](https://instadeepai.github.io/mlip/user_guide/training.html#io-handling-and-logging) part in our deep-dive on logging for more information. The code below adds a local directory for checkpointing to the I/O handler, which activates checkpointing during training.

In [ ]:
# io_handler = TrainingIOHandler(
#     TrainingIOHandler.Config(
#         local_model_output_dir="training/model_training"
#     )
# )

Next, we can **attach logging functions to the I/O handler**:

Users can attached as many logging functions as required to the I/O handler. In this example, we attach two.
1. The [`log_metrics_to_line`](https://instadeepai.github.io/mlip/api_reference/training/training_io_handling.html#mlip.training.training_loggers.log_metrics_to_line) function that is also included in the default I/O handler that we used in the previous example.
2. A custom function that just keeps track of the validation set losses, so we can later easily create a curve from it.

In [ ]:
# # The following logger is also attached in the default I/O handler
# # # that was used in the training above
# io_handler.attach_logger(log_metrics_to_line)

# # Define a custom logging function that keeps track of validation loss
# validation_losses = []
# training_losses = []
# def _custom_logger(category, to_log, epoch_number):
#   if category == LogCategory.EVAL_METRICS:
#     validation_losses.append(to_log["loss"])
#   elif category == LogCategory.TRAIN_METRICS:
#     training_losses.append(to_log["loss"])

# # Attach our custom logging function to the I/O handler
# io_handler.attach_logger(_custom_logger)

The custom logging function is called several times during the training loop with the argument `category` (e.g. `TRAIN_METRICS`, `EVAL_METRICS`) telling the function what is currently being logged, for example, train or evaluation metrics. It is of enum type [`LogCategory`](https://instadeepai.github.io/mlip/api_reference/training/training_io_handling.html#mlip.training.training_io_handler.LogCategory). See the documentation of the built-in function [`log_metrics_to_line`](https://instadeepai.github.io/mlip/api_reference/training/training_io_handling.html#mlip.training.training_loggers.log_metrics_to_line) for what we expect the logging function's signature to be.

Finally, we can **create our training loop**:

At a minimum, it needs as input:
- a training dataset
- a validation dataset
- a force field
- a loss
- an optimizer
- a config (which specifies for instace the number of epochs)

Note that the config is documented [here](https://instadeepai.github.io/mlip/api_reference/training/training_loop_config.html). Its only argument that lacks a default value is the number of epochs to train for.

In [ ]:
# training_config = TrainingLoop.Config(
#     num_epochs=run.config["epochs"],
#     energy_weight=run.config["energy_weight"],
#     force_weight=0, # Reduce this to make energy more important
# )

# training_loop = TrainingLoop(
#     train_dataset=train_set,
#     validation_dataset=validation_set,
#     force_field=force_field,
#     loss=loss,
#     optimizer=optimizer,
#     config=training_config,
#     io_handler=io_handler,
# )

## 3. Running a training loop

**Running the loop**:

The following box runs the prepared training loop. Note that training will be a **lot more efficient for GPU users** (depending on the GPU, one should expect ~1s to ~12s per epoch, once the code is compiled) - for CPU users our measures ranged from ~12s to ~100s per epoch.

In [ ]:
# training_loop.run()

We can now **access the information stored** by the custom logger saved into our validation loss list:

In [ ]:
# print(validation_losses)

Let's create a training curve from these values.

In [ ]:
# epoch_nums = list(range(len(validation_losses)))
# epoch_nums = list(range(len(training_losses)))
# plt.plot(epoch_nums, training_losses, c="red")
# plt.plot(epoch_nums, validation_losses, c="blue")
# plt.xlabel("Epoch")
# plt.ylabel("Validation loss (blue) & Training Loss (red)")
# plt.xticks(epoch_nums)
# plt.show()

**Recovering the best validation model**:

After training has completed, the [`TrainingLoop`](https://instadeepai.github.io/mlip/api_reference/training/training_loop.html) holds all the relevant information about the run. We can obtain the force field with the best validation parameters as follows:

In [ ]:
best_force_field = training_loop.best_model

This force field object can now be applied in, for example, MD simulations or energy minimizations.

Furthermore, these checkpoints can of course also be **used to restart a training from a given checkpoint**. We refer to the [documentation of the I/O handler's config](https://instadeepai.github.io/mlip/api_reference/training/training_io_handling.html#mlip.training.training_io_handler.TrainingIOHandlerConfig) for more information on this.

**Saving the model to a zip file**:

We can also save the trained model in zip format. This is also the format that we provide our pre-trained models in.

In [ ]:
save_model_to_zip("training/my_final_model.zip", best_force_field)

**Loading a pre-trained model**

We can also load a pre-trained model from a zip file. This is useful if you want to use a pre-trained model for inference or fine-tuning on a different dataset.

In [ ]:
best_force_field = load_model_from_zip(Mace, "training/my_final_model.zip")

print("Dataset info:", best_force_field.dataset_info)

## Evaluation

The model will be tested on its ability to predict the energies of new conformations
of the same molecule. However, to test the generalization
capabilities of the model, these conformations are sampled at higher temperature,
i.e., 1200 Kelvin. The test conformations are located in the
`test_public.xyz` file. You can predict energies for them with a model saved in the
zip format with the mlip library's batched inference functionality, described
[here](https://instadeepai.github.io/mlip/user_guide/simulations.html#batched-inference)
in the mlip documentation or explained in section 2 of
[mlip's simulation tutorial](https://github.com/instadeepai/mlip/blob/main/tutorials/simulation_tutorial.ipynb).
The public leaderboard contains the target energies. The metric the predictions will be scored on is root-mean-square error (RMSE).


**Hint**: to get the best energies, you want to put higher weights on energies during training which is not the case in the default settings, where forces are more highly weighted.


In [ ]:
from ase.io import read as ase_read

test_data = "data/test_public.xyz"
structures = ase_read(test_data, index=":")

We can now run inference with a single pre-built function, note Jax starts by compiling all the required functions. It may appear slow at the beginning but this provides significant acceleration at scales (compilation is saved in the notebook kernel, so if you want an illustration of the speed gains, you can run the cell twice):

In [ ]:
from mlip.inference import run_batched_inference

predictions = run_batched_inference(structures, best_force_field, batch_size=8)

# Get your output into submission format

We need to get our outputs into their "camera-ready" form.

In [ ]:
energies = np.array([prediction.energy for prediction in predictions])

# Create DataFrame
df = pd.DataFrame({
    'ID': np.arange(len(energies)),
    'energies': energies
})

df

In [ ]:
your_name = "YOUR_NAME_HERE"
filename = f"{your_name}_submission.csv"
df.to_csv(filename, index=False)
print(f"Saved submission to {filename}")

Submit your solution by uploading the CSV file to the [Zindi competition page](https://zindi.africa/competitions/indabax-south-africa-2025-hackathon-with-instadeep).